# Logistic Regression example step to step

Demostrate Logistic Regression algorithm steps by a simple dataset.



In [34]:
import numpy as np
import tkinter
import matplotlib.pyplot as plt
import h5py
import scipy


#from PIL import Image
from scipy import ndimage
#from lr_utils import load_dataset

np.set_printoptions(precision=4)
#%matplotlib inline

**We will using this very simple dataset. Purposes of this notebook is learning how algorith calculates each step not for accuracy or performance of it**:

<table style="width:25%">
  <tr>
  <th>**m2**</th>
  <th>**rooms**</th>
  <th>**price > 100K**</th>
  </tr>
  <tr>
    <td> 100 </td>
    <td> 3 </td> 
    <td> 1 </td> 
  </tr>
  
  <tr>
    <td> 45 </td>
    <td> 2 </td>
    <td> 0 </td> 
  </tr>
  <tr>
    <td> 20 </td>
    <td> 1 </td>
    <td> 0 </td> 
  </tr>
  
</table>

In [4]:
#Original dataset
dataset=np.array([[100, 3, 1],[45, 2, 0],[20, 1, 0]])
#Extract trainset from this
train_set_x_orig = dataset[:,:2]
train_set_y = dataset[:,-1]

#CREATION OF VERY SIMPLE TEST SET
test_dataset = np.array([[90,3,1],[80,2,1],[30,1,0],])
test_set_x_orig = test_dataset[:,:2]
test_set_y = test_dataset[:,-1]

m_dataset, cols_dataset = dataset.shape
m_test_dataset, cols_test_dataset = dataset.shape
m_train, features_train = train_set_x_orig.shape
m_test, features_test = test_set_x_orig.shape
assert(m_dataset == m_train and features_train == cols_dataset - 1)
assert(m_train == m_test_dataset and features_test == cols_test_dataset - 1)

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Number of features: features_train = " + str(features_train))
print ("train_set_x_orig shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_orig shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))


Number of training examples: m_train = 3
Number of testing examples: m_test = 3
Number of features: features_train = 2
train_set_x_orig shape: (3, 2)
train_set_y shape: (3,)
test_set_x_orig shape: (3, 2)
test_set_y shape: (3,)


In [9]:
# Reshape the training and test examples
X_train = train_set_x_orig.reshape(m_train,-1).T
X_test = test_set_x_orig.reshape(m_test,-1).T

print ("X_train shape: " + str(X_train.shape))
print ("X_test shape: " + str(X_test.shape))
assert (X_train.shape[0] == features_train and X_train.shape[1] == m_train)
assert (X_test.shape[0] == features_test and X_test.shape[1] == m_test)
print ("result of reshaping: " )
print (str(train_set_x_flatten[:,:]))

X_train shape: (2, 3)
X_test shape: (2, 3)
result of reshaping: 
[[100  45  20]
 [  3   2   1]]


**For the scope of this example, we are not standarize dataset**

**define all intermediate functions (abstrac from it until understand manual calculation)**:

- def sigmoid(z):
    - Compute the sigmoid of z
      Arguments:
         z -- A scalar or numpy array of any size.
      Return:
         s -- sigmoid(z)
    
- def initialize_with_zeros(dim):
    - This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
        Argument:
          dim -- size of the w vector we want (or number of parameters in this case)
          
        Returns:
          w -- initialized vector of shape (dim, 1)
          b -- initialized scalar (corresponds to the bias)

- def propagate(w, b, X, Y):
    - Implement the cost function and its gradient for the propagation explained above
      Arguments:
        w -- weights, a numpy array of size (num_px * num_px * 3, 1)
        b -- bias, a scalar
        X -- data of size (num_px * num_px * 3, number of examples)
        Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)
      Return:
        cost -- negative log-likelihood cost for logistic regression
        dw -- gradient of the loss with respect to w, thus same shape as w
        db -- gradient of the loss with respect to b, thus same shape as b

- def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
     - This function optimizes w and b by running a gradient descent algorithm
       Arguments:
        w -- weights, a numpy array of size (num_px * num_px * 3, 1)
        b -- bias, a scalar
        X -- data of shape (num_px * num_px * 3, number of examples)
        Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
        num_iterations -- number of iterations of the optimization loop
        learning_rate -- learning rate of the gradient descent update rule
        print_cost -- True to print the loss every 100 steps
      Returns:
        params -- dictionary containing the weights w and bias b
        grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
        costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
- def predict(w, b, X):
    - Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
      Arguments:
        w -- weights, a numpy array of size (num_px * num_px * 3, 1)
        b -- bias, a scalar
        X -- data of size (num_px * num_px * 3, number of examples)
      Returns:
        Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    

In [32]:
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

def initialize_with_zeros(dim):
    w = np.zeros((dim,1))
    b = 0

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

def propagate(w, b, X, Y):
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    print("WT * X:")
    print(np.dot(w.T, X))
    print("bias: ", b)
    A = sigmoid(np.dot(w.T, X) + b)             # compute activation
    print("A:")
    print(A)
    print("Y:")
    print(Y)
    cost = np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A)) / np.negative(m)    # compute cost
        
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = (np.dot(X,(A - Y).T))/m
    db = (np.sum(A - Y))/m
    
    print("dw:")
    print(dw)
    print("db:")
    print(dw)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% iter nº {} %%%%%%%%%%%%%%%%%%%%%%%5".format(i))

        grads, cost = propagate(w, b, X, Y)

        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        print("Acum w:")
        print(w)
        print("Acum b:")
        print(b)
        
        # Record the costs
        #if i % 100 == 0:
        costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost: #and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

def predict(w, b, X):
    
    print("w shape:", w.shape)
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    print("X shape", X.shape)
    print("w reshape", w.shape)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = sigmoid(np.dot(w.T, X) + b)

    
    print("A shape: ", A.shape)
    print("Y shape ", Y_prediction.shape)
    
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]

        if A[0,i]>=0.5:
            Y_prediction[0,i] = 1.0
        else:
            Y_prediction[0,i] = 0.0

    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

### Logistic regression algorithm

- Get X

- Initialize de parameters w and b to zero.

- Iterate to find minimun of cost **`for i:=1 to num_iterations repeat:`** 

    - Compute: 
        $$A = \sigma(w^T X + b) = (a^{(0)}, a^{(1)}, ..., a^{(m-1)}, a^{(m)})$$

    - `For each` training set calculate the loss 
        $$L(a^i,y^i) = -[y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})] $$

    - Calculate the cost function: 
        $$J = -\frac{1}{m}\sum_{i=1}^{m}L(a^i,y^i)$$
    
    - Calculate the derivatives of cost function respect w and b
        $$ dw=\frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T\tag{1}$$
        $$ db=\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{2}$$
    
    - Update de parameters w and b, the update rule is ($\alpha$ is the learning rate):
        $$ w = w - \alpha dw $$
        $$ b = b - \alpha db $$




In [33]:
#LOGISTIC REGRESSION ALGORITHM

#initialize parameters with zeros (≈ 1 line of code)
w, b = initialize_with_zeros(features_train)

num_iterations = 2
learning_rate = 0.01
print_cost = True
parameters, grads, costs = optimize(w, b, train_set_x_flatten, train_set_y, num_iterations, learning_rate, print_cost)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% iter nº 0 %%%%%%%%%%%%%%%%%%%%%%%5
WT * X:
[[ 0.  0.  0.]]
bias:  0
A:
[[ 0.5  0.5  0.5]]
Y:
[1 0 0]
dw:
[[-5.83333333]
 [ 0.        ]]
db:
[[-5.83333333]
 [ 0.        ]]
Acum w:
array([[ 0.05833333],
       [ 0.        ]])
Acum b:
-0.0016666666666666666
Cost after iteration 0: 0.693147
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% iter nº 1 %%%%%%%%%%%%%%%%%%%%%%%5
WT * X:
[[ 5.83333333  2.625       1.16666667]]
bias:  -0.00166666666667
A:
[[ 0.99707539  0.93234826  0.76224005]]
Y:
[1 0 0]
dw:
[[ 18.96933739]
 [  0.87272092]]
db:
[[ 18.96933739]
 [  0.87272092]]
Acum w:
array([[-0.13136004],
       [-0.00872721]])
Acum b:
-0.0073055456906058851
Cost after iteration 1: 1.377602
